## **Import Library**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# นำเข้า Method สำหรับอ่าน และแสดงผลภาพจาก skimage.io 
from skimage.io import imread

# นำเข้า Library matplotlib
import matplotlib.pyplot as plt

# นำเข้า method สำหรับแบ่งข้อมูล Train และ Test Model sklearn.model_selection
from sklearn.model_selection import train_test_split

# นำเข้า method สำหรับประเมิณความแม่นยำของ Model จาก sklearn.metrics
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# นำเข้า Library PyTorch และ modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

# นำเข้า Library Pandas
import pandas as pd

# นำเข้า Library Numpy
import numpy as np

# นำเข้า Library OpenCV
import cv2

## **Data Loader**

In [ ]:
train_img = []
train_label = []

# นำเข้าข้อมูลรูปภาพที่ใช้ทำการฝึกสอน
for floder in tqdm(range(1,7)):
  for i in range(1, 21):
    impath = '/content/drive/MyDrive/Colab Notebooks/IMG/Tr/' + str(floder) + '/text (' + str(i) + ').bmp'

    # อ่านข้อมูลรูปภาพ และแปลงให้เป็นภาพ Gray scale
    img = imread(impath, as_gray=True)

    # ทำการปรับขนาด Pixel ของภาพ 
    img = cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA)

    # แปลง Data type ให้เป็น float32
    img = img.astype('float32')

    # ปรับค่าของ Pixel ให้อยู่ในขนาด 0-1
    img /= 255.0

    # ทำการข้อมูลรูปภาพไว้ในตัวแปร train_img
    train_img.append(img)

    # สร้าง Label ตามจำนวนของ Floder
    train_label.append(floder)

# ทำการแปลงข้อมูลให้อยู่ใน format ของ numpy เพื่อให้ทำการคำนวณได้เร็วขึ้น
train_x = np.array(train_img)
train_y = np.array(train_label)

# ทำการเช็คขนาดของข้อมูลที่อยู่ใน ตัวแปร train_x, train_y  
print(train_x.shape, train_y.shape)

100%|██████████| 6/6 [00:00<00:00, 20.85it/s]

(120, 128, 128) (120,)


## **Training & Validating Set Generation**

In [ ]:
# ทำการแบ่งข้อมูลสำหรับ Train 90% และ Test 10%
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1)

# ทำการเช็คขนาดของข้อมูลที่อยู่ใน ตัวแปร train_x, val_x 
print(train_x.shape, val_x.shape)

(108, 128, 128) (12, 128, 128)


In [ ]:
# converting training images into torch format
train_x = train_x.reshape(108, 1, 128, 128)
train_x = torch.from_numpy(train_x).to(torch.float32)

# converting the target into torch format
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y).to(torch.float32)

# shape of training data
print(train_x.shape, train_y.shape)

# converting validation images into torch format
val_x = val_x.reshape(12, 1, 128, 128)
val_x = torch.from_numpy(val_x).to(torch.float32)

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y).to(torch.float32)

# shape of validation data
print(val_x.shape, val_y.shape)

torch.Size([108, 1, 128, 128]) torch.Size([108])
torch.Size([12, 1, 128, 128]) torch.Size([12])


## **Model**

In [ ]:
class Net(Module):
    def __init__(self): # ก ําหนดโครงสร้ํางใน Constructor

      super(Net, self).__init__()
      self.cnn_layers = Sequential(
        # Defining a 2D convolution layer
        Conv2d(1,12,kernel_size=3, stride=1, padding=1),
        BatchNorm2d(12),
        ReLU(inplace=True),
        MaxPool2d(kernel_size=2, stride=2),

        # Defining another 2D convolution layer
        Conv2d(12,24,kernel_size=3, stride=1, padding=1),
        BatchNorm2d(24),
        ReLU(inplace=True),
        MaxPool2d(kernel_size=2, stride=2),

        # Defining another 2D convolution layer
        Conv2d(24, 36, kernel_size=3, stride=1, padding=1),
        BatchNorm2d(36),
        ReLU(inplace=True),
        MaxPool2d(kernel_size=2, stride=2),
      )

      self.drop_out = Dropout()
      self.fc1 = Linear(36 * 16 * 16, 1000)
      self.fc2 = Linear(1000, 500)
      self.fc3 = Linear(500, 7)

    # Defining how the data flows through these layers when performing the forward pass through the network
    def forward(self, x):
      x = self.cnn_layers(x)
      # convert feature map to vector form
      x = x.view(x.size(0), -1)
      x = self.drop_out(x)
      x = self.fc1(x)
      x = self.drop_out(x)
      x = self.fc2(x)
      x = self.fc3(x)
      return x

# model = Net()
# torch.save(model, '/content/drive/MyDrive/Colab Notebooks/IMG/Net.py')

## **Defining Learning Algorithm**

In [ ]:
# defining the model
model = Net()

# defining the optimizer
optimizer = SGD(model.parameters(), lr=0.07, momentum=0.9)

# # defining the loss function
criterion = CrossEntropyLoss()

# checking if GPU is available
if torch.cuda.is_available():
  model = model.cuda()
  criterion = criterion.cuda()

## **Training Model**

In [ ]:
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# defining the number of epochs
n_epochs = 20
# training the model
for epoch in tqdm(range(n_epochs)):
  model.train()
  tr_loss = 0

  # getting the training set
  x_train, y_train = Variable(train_x), Variable(train_y)

  # getting the validation set
  x_val, y_val = Variable(val_x), Variable(val_y)

  # clearing the Gradients of the model parameters
  optimizer.zero_grad()

  # prediction for training and validation set
  output_train = model(x_train)
  output_val = model(x_val)

# computing the training and validation loss
# we convert the results because they aren't in the good format

y_train = y_train.long()
y_train = y_train.squeeze_()
y_val = y_val.long()
y_val = y_val.squeeze_()

loss_train = criterion(output_train, y_train)
loss_val = criterion(output_val, y_val)
train_losses.append(loss_train)
val_losses.append(loss_val)

# computing the updated weights of all the model parameters
loss_train.backward()
optimizer.step()
tr_loss = loss_train.item()

100%|██████████| 20/20 [00:13<00:00,  1.49it/s]


In [ ]:
# Saving Entire Model
# A common PyTorch convention is to save models
# using either a .pt or .pth file extension.
# torch.save(model, '/content/drive/MyDrive/Colab Notebooks/IMG/model91.67%.pt')

## **Testing**

In [ ]:
# prediction for validation set
# don’t need gradients in test step since the
# parameter updates has been done in training
# step. Using ‘torch.no_grad()’ in the test and
# validation phase yields the faster
# inference(speed up computation) and reduced
# memory usage(which allows us to use larger
# size of batch).

with torch.no_grad():
  output = model(val_x)

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
print("{:.2f} %".format(round(accuracy_score(val_y, predictions) * 100, 3)))

83.33 %


## **Executing Model with Testing Image**

In [ ]:
# # loading test images
# test_img = []

# # Test Image Loader and Feature Extraction
# impath = '/content/drive/MyDrive/Colab Notebooks/IMG/Tr/6/text (20).bmp'

# # reading the image
# img = imread(impath, as_gray=True)

# # RESIZE
# img = cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA)

# # normalizing the pixel values AND converting the type of pixel to float
# img = img.astype('float32')
# img /= 255.0

# # appending the image into the list
# test_img.append(img)

# # converting the list to numpy array
# test_x = np.array(test_img)

# # converting training images into torch format
# test_x = test_x.reshape(1, 1, 128, 128)
# test_x = torch.from_numpy(test_x).to(torch.float32)

# # generating predictions for test set
# with torch.no_grad():
#   #output = model(test_x.cuda())
#   output = model(test_x)

# softmax = torch.exp(output).cpu()
# prob = list(softmax.numpy())
# predictions = np.argmax(prob, axis=1)
# print(predictions)

[2]
